### Install dependencies

In [95]:
#!pip install NetworkX
#!pip install biopython
#!pip install blosum

### Imports

In [96]:
from Bio.Blast.Applications import NcbiblastpCommandline
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import uuid
from Bio.Seq import Seq
import blosum as bl
import re
import pandas as pd
import numpy as np
import tqdm
import torch
import pandas as pd
from Bio import SeqIO
import networkx as nx

### Extract data

In [97]:
# import tarfile
  
# # open file
# file = tarfile.open('temp_outs.tar.gz')
  
# # extracting file
# file.extractall('./')
  
# file.close()

### Align sequences with ESM (code mostly from Mahdi)

In [98]:
seq_2 = torch.load(f"temp_outs\outputs\QBZ81690.1.pt")
seq_1 = torch.load(f"temp_outs\outputs\AGO83968.2.pt")
seq_1

{'label': 'AGO83968.2',
 'representations': {36: tensor([[ 0.0292, -0.3383, -0.0874,  ..., -0.1267, -0.0776, -0.0283],
          [ 0.1408,  0.0147, -0.0469,  ..., -0.1412, -0.2966,  0.1156],
          [ 0.0775, -0.1473, -0.0884,  ..., -0.0069, -0.0825,  0.0087],
          ...,
          [-0.0428, -0.0048, -0.0294,  ..., -0.0273, -0.0917, -0.0409],
          [-0.1028,  0.0238, -0.0095,  ..., -0.0699, -0.1929,  0.1020],
          [-0.0754, -0.0673, -0.1764,  ...,  0.0654, -0.1087,  0.0849]])},
 'mean_representations': {36: tensor([-0.0061, -0.0293,  0.0106,  ..., -0.0063, -0.0427, -0.0066])}}

In [99]:
# Compute all pairwise distances
data = []
for i in range(seq_1["representations"][36].shape[0]):
    line = []
    for j in range(seq_2["representations"][36].shape[0]):
        v1 = seq_1["representations"][36][i]
        v2 = seq_2["representations"][36][j]
        line.append(round(torch.cdist(v1.unsqueeze(0), v2.unsqueeze(0)).item(), 2))
    data.append(line)
    

# Write the matrix to the excel sheet    
data = pd.DataFrame(data)
data

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
0,4.85,7.78,7.28,7.18,6.94,7.20,7.01,7.07,7.05,7.19,...,7.14,6.91,6.86,6.90,6.99,6.73,7.01,6.74,7.12,7.21
1,7.74,7.29,7.33,7.67,7.04,7.38,7.56,7.59,7.22,7.24,...,7.68,7.52,7.29,7.49,7.62,7.26,7.47,7.25,7.52,7.60
2,7.44,7.96,7.06,7.11,6.88,7.25,6.90,7.08,7.38,7.55,...,7.11,7.08,6.96,7.04,6.96,6.89,7.12,6.88,7.38,7.45
3,7.62,8.19,7.43,7.23,7.00,7.58,7.17,6.72,7.57,7.94,...,7.23,7.25,7.27,7.20,7.15,7.00,7.47,7.10,7.54,7.50
4,7.52,7.94,6.92,7.23,6.83,6.83,7.02,6.94,6.55,6.88,...,7.06,6.92,6.13,6.98,7.03,6.54,6.89,6.58,7.10,7.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,6.24,7.08,6.07,4.80,5.53,5.86,5.64,5.50,5.53,5.67,...,5.50,5.34,5.14,3.68,4.96,4.84,5.29,4.91,5.49,5.91
104,6.15,6.90,5.90,5.86,5.38,5.67,5.56,5.49,4.61,5.68,...,5.26,5.13,4.88,5.11,4.93,3.61,5.10,3.84,5.29,5.80
105,6.01,6.80,5.94,5.65,5.39,5.58,5.31,5.30,5.37,5.45,...,5.16,4.99,4.80,5.02,4.92,4.52,4.82,4.44,4.96,5.67
106,6.34,7.03,6.17,6.08,5.81,5.01,4.69,5.62,5.70,5.92,...,4.52,5.48,5.26,5.40,5.28,5.04,5.43,4.80,5.08,5.63


In [100]:
seqs = SeqIO.to_dict(SeqIO.parse("test/ab.fa", 'fasta'))

seq_ids = list(seqs.keys())
seq_ids = [seq_ids[1]] + [seq_ids[0]]
seq_ids

['AGO83968.2', 'QBZ81690.1']

In [101]:
top_n = 5
rows = pd.DataFrame({n: data.T[col].nsmallest(top_n).index.tolist() 
                  for n, col in enumerate(data.T)}).T

cols = pd.DataFrame({n: data[row].nsmallest(top_n).index.tolist() 
                  for n, row in enumerate(data)})

matches = set()
for i, n_cols in enumerate(rows.values):
    for c in n_cols:
        if i in cols.iloc[:, c].values:
            matches.add((i, int(c)))

temp_set = set()
for match in matches:
    if match[0] > 0 and match[1] > 0 and (seqs[seq_ids[0]][match[0]-1] ==  seqs[seq_ids[1]][match[1]-1]):
            temp_set.add((match[0]-1, match[1]-1))
            #if sheet[(match[0]+1, match[1]+1)].color is None:
            #    sheet[(match[0]+1, match[1]+1)].color = "#82C341"
    if match[0] < len(seqs[seq_ids[0]]) - 1  and match[1] < len(seqs[seq_ids[1]])-1 and  (seqs[seq_ids[0]][match[0]+1] ==  seqs[seq_ids[1]][match[1]+1]):
            temp_set.add((match[0]+1, match[1]+1))
            #if sheet[(match[0]+1, match[1]+1)].color is None:
            #    sheet[( match[0]+3, match[1]+3)].color = "#82C341"

matches = matches.union(temp_set)
list(matches)[0:10]

[(52, 40),
 (90, 33),
 (67, 77),
 (54, 77),
 (104, 96),
 (98, 55),
 (89, 77),
 (64, 32),
 (18, 44),
 (55, 93)]

In [102]:
def build_paths_graph(matches):
    graph = nx.DiGraph()
    dag = {}
    max_depth = max([x[0] for x in matches])
    sorted_matches = sorted(matches, key=lambda x: x[1])
    for i in range(len(sorted_matches) - 1):
        last_depth =  max_depth
        dag[sorted_matches[i]] = []
        #print(f"Comparing {sorted_matches[i]} and {sorted_matches[j]}")
        for j in range(i+1, len(sorted_matches)):
            if (sorted_matches[j][0]) < last_depth and  (sorted_matches[j][0] >  sorted_matches[i][0]):
                dag[sorted_matches[i]].append(sorted_matches[j])
                seq_1_idx, seq_2_idx = sorted_matches[j]
                graph.add_edge(sorted_matches[i], sorted_matches[j], weigth=data.iloc[seq_1_idx, seq_2_idx])
                last_depth = sorted_matches[j][0]
    return graph


graph = build_paths_graph(matches)
esmAlignment = nx.dag_longest_path(graph)
#len(esmAlignment)
esmAlignment[0:10]

[(0, 0),
 (13, 9),
 (14, 10),
 (24, 10),
 (25, 11),
 (27, 12),
 (33, 13),
 (34, 18),
 (44, 18),
 (47, 32)]

### Align sequences with BLAST

In [103]:
seqs = SeqIO.to_dict(SeqIO.parse("test/ab.fa", 'fasta'))
str(seqs['QBZ81690.1'].seq)

'MGQANTTIKGYSAVSQDALFATYGITEGDYQAKASAAIERIRAMPEGYASPEDRAAAINAIRTGTCGDDTELLTRVRAALDRWQRDCASTGHALKSKVV'

In [104]:
data = ['', 'QBZ81690.1', '', 'AGO83968.2']

seq2 = SeqRecord(seqs[data[1]].seq,
                 id=seqs[data[1]].id)
seq1 = SeqRecord(seqs[data[3]].seq,
                 id=seqs[data[3]].id)
SeqIO.write(seq1, f"seq_1.fasta", "fasta")
SeqIO.write(seq2, f"seq_2.fasta", "fasta")

str(seq1.seq)

'MNFIHKILGTNRAGYAALPPCAVEMSCAEESPNVLDPSSHEYAELKEKTIRIIVALPTQYLSADKGRAAIEAIRSGEPGREALYDEVWDAAMAYSRDIAAGRAAKPTR'

In [105]:
output = NcbiblastpCommandline(query=f"seq_1.fasta", subject=f"seq_2.fasta", outfmt="6 qstart qseq sstart sseq")()[0]
if len(output) < 2:
    output = f"{seqs[data[1]].id}\t{seqs[data[3]].id}\t NO_HIT"
#output = output.split("\n")
print(output)

41	EYAELKEKTIRIIVALPTQYLSADKGRAAIEAIRSGEPGREA-LYDEVWDAAMAYSRDIAA	29	DYQAKASAAIERIRAMPEGYASPEDRAAAINAIRTGTCGDDTELLTRVRAALDRWQRDCAS
80	REALY	16	QDALF



In [106]:
def GetBlastIndices(start, blast):
    indices = [i for i in range(start, start + len(blast))]
    
    gaps = [i for i, c in enumerate(blast) if c == "-"]
    
    for gap in gaps:
        indices.insert(gap, -1)
        
    return indices[0:len(blast)]

GetBlastIndices(2, "abcd-efg")

[2, 3, 4, 5, -1, 6, 7, 8]

In [107]:
def GetBlastAlignRegion(start1, blast1, start2, blast2):
    start1 = int(start1)
    start2 = int(start2)
    
    i1 = GetBlastIndices(start1, blast1)
    i2 = GetBlastIndices(start2, blast2)
    
    return list(zip(i1, i2))

testBlastIndices = GetBlastAlignRegion(2, "abcde", 1, "wxy-z")
testBlastIndices

[(2, 1), (3, 2), (4, 3), (5, -1), (6, 4)]

In [108]:
def BlastAlign(file1, file2):
    blastRaw = NcbiblastpCommandline(query=file1, subject=file2, outfmt="6 qstart qseq sstart sseq")()[0]
    blastLines = blastRaw.splitlines()
    blastFields = [re.split(r'\t+', line) for line in blastLines]
    
    alignments = [GetBlastAlignRegion(*blast) for blast in blastFields]
    
    return alignments
    
blastAlignments = BlastAlign(f"seq_1.fasta", f"seq_2.fasta")
#blastAlignments
[seq[40:45] for seq in blastAlignments]

[[(81, 69), (82, 70), (-1, 71), (83, 72), (84, 73)], []]

### Combine the results

In [109]:
def InterweaveAlignments(align1, align2):
    (indices11, indices12) = zip(*align1)
    (indices21, indices22) = zip(*align2)
    
    df1 = pd.DataFrame(data={"QLoc": indices11, "SLoc": indices12})
    #df1.set_index(keys=["QLoc", "SLoc"], inplace=True)
    
    df2 = pd.DataFrame(data={"QLoc": indices21, "SLoc": indices22})
    #df2.set_index(keys=["QLoc", "SLoc"], inplace=True)
    
    df = pd.merge(df1, df2, on=["QLoc", "SLoc"], how="outer", indicator=True, sort=True)
    #df.set_index(keys=["QLoc", "SLoc"], inplace=True)
    
    #split the indicator into bool columns
    df = pd.get_dummies(df, columns=["_merge"], prefix="")
    df["Align1"] = (df["_left_only"] | df["_both"]).astype(bool)
    df["Align2"] = (df["_right_only"] | df["_both"]).astype(bool)
    df = df.drop(columns=["_left_only","_right_only","_both"])
    
    return df
    
testAligns = InterweaveAlignments(blastAlignments[0], esmAlignment)
testAligns[50:60]

,QLoc,SLoc,Align1,Align2
50,75,63,True,True
51,76,64,True,True
52,77,65,True,True
53,78,66,True,True
54,79,67,True,False
55,80,68,True,False
56,81,69,True,False
57,82,70,True,False
58,83,72,True,False
59,84,68,False,True


In [110]:
#Identify the positive matches by looking directly in the score matrix? https://www.ncbi.nlm.nih.gov/Class/FieldGuide/BLOSUM62.txt
blosumDict = dict(bl.BLOSUM(62))

def Blosum(a, b):
    key = str(a.upper()) + str(b.upper())
    return blosumDict.get(key, np.nan)

Blosum("A", "N")

-2.0

In [111]:
def CompareAlignments(query, subject, align1, align2):
    df = InterweaveAlignments(align1, align2)
    df["Query"] = [query[i] if (i >= 0) else "-" for i in df.QLoc]
    df["Subject"] = [subject[i] if (i >= 0) else "-" for i in df.SLoc]
    df["Score"] = [Blosum(a,b) for (a,b) in list(zip(df.Query, df.Subject))]
    
    return df

testAligns = CompareAlignments(seq1.seq, seq2.seq, blastAlignments[0], esmAlignment)
testAligns[50:60]

,QLoc,SLoc,Align1,Align2,Query,Subject,Score
50,75,63,True,True,G,G,6.0
51,76,64,True,True,E,T,-1.0
52,77,65,True,True,P,C,-3.0
53,78,66,True,True,G,G,6.0
54,79,67,True,False,R,D,-2.0
55,80,68,True,False,E,D,2.0
56,81,69,True,False,A,T,0.0
57,82,70,True,False,L,E,-3.0
58,83,72,True,False,Y,L,-1.0
59,84,68,False,True,D,D,6.0


In [112]:
alignmentDFs = [CompareAlignments(seq1.seq, seq2.seq, blastAlign, esmAlignment) for blastAlign in blastAlignments]
alignmentDFs[0]

,QLoc,SLoc,Align1,Align2,Query,Subject,Score
0,-1,71,True,False,-,L,NaN
1,0,0,False,True,M,M,5.0
2,13,9,False,True,G,G,6.0
3,14,10,False,True,Y,Y,7.0
4,24,10,False,True,M,Y,-1.0
...,...,...,...,...,...,...,...
78,100,89,True,False,G,T,-2.0
79,100,90,False,True,G,G,6.0
80,103,92,False,True,A,A,4.0
81,104,94,False,True,K,K,5.0


### Render the comparison

In [113]:
def SeqString(conditions, values, blank=" "):
    chars = np.select([conditions, ~conditions], [values, blank])
    return "".join(chars)

SeqString(pd.Series([False, True, True, False, True]), [1, 2, 3, 4, 5])

' 23 5'

In [114]:
def RenderAlignments(df):
    positive = pd.DataFrame(np.where(df.Score > 0.0, "+", None))
    perfect = pd.DataFrame(np.where(df.Query == df.Subject, "*", None))
    scorestring = perfect.combine_first(positive)[0].fillna(" ")
    
    a1_condition = [df.Align1, ~df.Align1]
    a2_condition = [df.Align2, ~df.Align2]
    
    #alignment 1
    print(SeqString(df.Align1, df.Query))
    print(SeqString(df.Align1, scorestring))
    print(SeqString(df.Align1, df.Subject))
    
    #anchors
    print(SeqString(df.Align1&df.Align2, "|"))
    
    #alignment 2
    print(SeqString(df.Align2, df.Query))
    print(SeqString(df.Align2, scorestring))
    print(SeqString(df.Align2, df.Subject))
    
RenderAlignments(alignmentDFs[0])

-        YAE LKE KTIR IIVALPTQYLSAD KG R A AIEAIRSGEPGREALY DEVWD AAM AYSRDIAAG    
         *         *   * *+*  * * +      * ** ***+*  * +      *   *    + ** *+     
L        YQA KAS AAIE RIRAMPEGYASPE DR A A AINAIRTGTCGDDTEL TRVRA ALD RWQRDCAST    
                   |  || | | || |   |       ||||||||||        |        ||||||      
 MGYMSAVL   L   K  I RII A P QY S  DK R A A IEAIRSGEPG     D  V  A   A YSRDIA  GAKP
 *** ****   *   *  * * * * *  * *  *  * * * * ***+*  *     *  *  *   * + ** *  *** 
 MGYYSAVL   L   K  I RRI A P GY S  DD R A A INAIRTGTCG     D  V  A   A WQRDCA  GAKS


In [115]:
for i, alignments in enumerate(alignmentDFs):
    print(f"\n\nBLAST HSP #{i+1} vs ESM alignment\n")
    RenderAlignments(alignments)



BLAST HSP #1 vs ESM alignment

-        YAE LKE KTIR IIVALPTQYLSAD KG R A AIEAIRSGEPGREALY DEVWD AAM AYSRDIAAG    
         *         *   * *+*  * * +      * ** ***+*  * +      *   *    + ** *+     
L        YQA KAS AAIE RIRAMPEGYASPE DR A A AINAIRTGTCGDDTEL TRVRA ALD RWQRDCAST    
                   |  || | | || |   |       ||||||||||        |        ||||||      
 MGYMSAVL   L   K  I RII A P QY S  DK R A A IEAIRSGEPG     D  V  A   A YSRDIA  GAKP
 *** ****   *   *  * * * * *  * *  *  * * * * ***+*  *     *  *  *   * + ** *  *** 
 MGYYSAVL   L   K  I RRI A P GY S  DD R A A INAIRTGTCG     D  V  A   A WQRDCA  GAKS


BLAST HSP #2 vs ESM alignment

                                  EALYD              
                                  +**+               
                                  DALFA              
                                                     
MGYMSAVLLKIRIIAPQYSDKRAAIEAIRSGEPG     DVAAYSRDIAGAKP
*** ******** *** *** **** ***+*  *     ****+ ** **** 
MGYYSAVLLKIRRIAPGYSDDR

### To Do: make sure gaps in Query sequence aren't shifted to the start of the alignment (same would happen for gaps in Subject too, but would only shift before any pairs with the same QLoc)